In [45]:
 #%pip install sagemaker  --upgrade --quiet 

In [46]:
#%pip install cufflinks termcolor 

In [47]:
""" Ciudad Segura MX ETL classifier and recognition

Notebook creado con la finalidad de procesar los datos de los delitos, 
hacer una clasificacion de delitos y hacer un match de los retratos para reconocimiento facial. 

Este Notebook se corre en Sagemaker, obtiene los datos crudos de S3, y exporta la data limpia a S3
    
"""

' Ciudad Segura MX ETL classifier and recognition\n\nNotebook creado con la finalidad de procesar los datos de los delitos, \nhacer una clasificacion de delitos y hacer un match de los retratos para reconocimiento facial. \n\nEste Notebook se corre en Sagemaker, obtiene los datos crudos de S3, y exporta la data limpia a S3\n    \n'

## LIBRERIAS

In [48]:
import pathlib
#bibliotecas
import sys
import pandas as pd
from IPython.core.display import display
pd.options.mode.chained_assignment = None
from scipy.stats import chisquare
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from statistics import mode
import numpy as np
import nltk
from sklearn.decomposition import PCA
import random
import re
import boto3
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

### Importación de librería personal

In [49]:
#Importando módulo con funciones a utilizar
sys.path.append('../librerias')
#from funciones import *
from limpieza_datos import *


## Lectura de dataset desde S3

In [50]:
s3 = boto3.resource('s3')

bucket_name = "itam-analytics-alberto"
bucket_prefix = "delitos/rawData/csv"
bucket = s3.Bucket(bucket_name)
prefix_objs = bucket.objects.filter(Prefix= bucket_prefix )

# Se toman del S3 todos los años(2019 -2023) y se van concatenando en un mismo dataframe 
df = pd.DataFrame()
for obj in prefix_objs:
    try:
        key = obj.key
        temp = pd.read_csv(obj.get()['Body'])     
        df =pd.concat([df, temp])
    except:
        continue
df.head()

,Unnamed: 0,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,1.0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,...,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,NaN,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,2.0,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,...,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,NaN,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,3.0,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,...,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,NaN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,4.0,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,...,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,NaN,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,5.0,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,...,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,NaN,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


In [51]:
df.shape

(1038430, 23)

# Limpieza de datos 

In [52]:
# Ver el grado de valores faltantes para cada columna 
completitud(df)

,columna,total,completitud
0,municipio_hechos,1028246,0.980711
1,Edad,366188,64.736381
2,Sexo,190025,81.700741
3,colonia_datos,73721,92.900725
4,Unnamed: 0,64344,93.803723
5,Categoria,64344,93.803723
6,fgj_colonia_registro,50410,95.145556
7,longitud,50204,95.165394
8,latitud,50202,95.165587
9,TipoPersona,6645,99.360092


In [53]:
#ELiminamos columnas con missings mayor al 40%
df =  df.drop(columns=['municipio_hechos'])
df = df.drop('Unnamed: 0', axis=1)

In [54]:
#Imputamos por media y moda para edad, sexo y colonia
df['Edad'] = df['Edad'].fillna(np.nanmean(df['Edad']))
df['Sexo'] = df['Sexo'].fillna(mode(df['Sexo']))
df['colonia_datos'] = df['colonia_datos'].fillna(mode(df['colonia_datos']))

In [55]:
#Eliminamos registros con pocos nan
df = df.dropna(subset=['colonia_datos', 'fgj_colonia_registro', 'longitud','latitud','TipoPersona','FechaHecho','Mes_hecho','Año_hecho','HoraHecho','HoraInicio','CalidadJuridica'])

In [56]:
# Verificar el grado de valores faltantes para cada columna 
completitud(df)

,columna,total,completitud
0,Categoria,60123,93.723621
1,idCarpeta,0,100.000000
2,fgj_colonia_registro,0,100.000000
3,colonia_datos,0,100.000000
4,alcaldia_hechos,0,100.000000
5,HoraInicio,0,100.000000
6,HoraHecho,0,100.000000
7,FechaHecho,0,100.000000
8,Mes_hecho,0,100.000000
9,Año_hecho,0,100.000000


In [57]:
df.shape

(957925, 21)

In [58]:
df.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Año_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.000000,FISICA,OFENDIDO,...,2018.0,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.000000,FISICA,VICTIMA Y DENUNCIANTE,...,2018.0,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.000000,FISICA,VICTIMA Y DENUNCIANTE,...,2018.0,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.000000,FISICA,VICTIMA Y DENUNCIANTE,...,2019.0,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,38.822344,FISICA,VICTIMA,...,2019.0,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


## Consistencia

In [59]:
# Ver que los tipos de las columnas sean las adecuadas 
df.dtypes

idCarpeta               float64
Año_inicio                int64
Mes_inicio               object
FechaInicio              object
Delito                   object
Categoria                object
Sexo                     object
Edad                    float64
TipoPersona              object
CalidadJuridica          object
competencia              object
Año_hecho               float64
Mes_hecho                object
FechaHecho               object
HoraHecho                object
HoraInicio               object
alcaldia_hechos          object
colonia_datos            object
fgj_colonia_registro     object
latitud                 float64
longitud                float64
dtype: object

In [60]:
# Eliminar todas las filas con valores de fecha fuera de los límites válidos
df = df[pd.to_datetime(df['FechaHecho'], errors='coerce').notnull()]

In [61]:
df['idCarpeta']=df['idCarpeta'].astype('Int64')
df['FechaInicio']=pd.to_datetime(df['FechaInicio'])
df['Edad']=df['Edad'].astype('int')
df['Año_hecho']=df['Año_hecho'].astype('Int64')
df['FechaHecho']=pd.to_datetime(df['FechaHecho'])


In [62]:
df.dtypes

idCarpeta                        Int64
Año_inicio                       int64
Mes_inicio                      object
FechaInicio             datetime64[ns]
Delito                          object
Categoria                       object
Sexo                            object
Edad                             int64
TipoPersona                     object
CalidadJuridica                 object
competencia                     object
Año_hecho                        Int64
Mes_hecho                       object
FechaHecho              datetime64[ns]
HoraHecho                       object
HoraInicio                      object
alcaldia_hechos                 object
colonia_datos                   object
fgj_colonia_registro            object
latitud                        float64
longitud                       float64
dtype: object

## Ingeniería de variables

In [63]:
df['dia_inicio'] = df['FechaInicio'].dt.day
df['dia_hecho'] = df['FechaHecho'].dt.day

In [64]:
df[['horas_hecho', 'minutos_hecho', 'segundos_hecho']] = df['HoraHecho'].str.split(':', expand=True)
df[['horas_inicio', 'minutos_inicio', 'segundos_inicio']] = df['HoraInicio'].str.split(':', expand=True)
df['horas_hecho'] = df['horas_hecho'].astype('int')
df['minutos_hecho'] = df['minutos_hecho'].astype('int')
df['segundos_hecho'] = df['segundos_hecho'].astype('int')
df['horas_inicio'] = df['horas_inicio'].astype('int')
df['minutos_inicio'] = df['minutos_inicio'].astype('int')
df['segundos_inicio'] = df['segundos_inicio'].astype('int')

In [65]:
df.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,latitud,longitud,dia_inicio,dia_hecho,horas_hecho,minutos_hecho,segundos_hecho,horas_inicio,minutos_inicio,segundos_inicio
0,8324429,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62,FISICA,OFENDIDO,...,19.36125,-99.18314,4,29,12,0,0,12,19,0
1,8324430,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,...,19.47181,-99.16458,4,15,15,0,0,12,20,0
2,8324431,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,...,19.33797,-99.18611,4,22,15,30,0,12,23,0
3,8324435,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,...,19.40327,-99.05983,4,4,6,0,0,12,27,0
4,8324438,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,38,FISICA,VICTIMA,...,19.35480,-99.06324,4,3,20,0,0,12,35,0


In [66]:
df['FechaHecho'] = pd.to_datetime(df['FechaHecho'],format='%Y-%m-%d')
df['FechaInicio'] = pd.to_datetime(df['FechaInicio'],format='%Y-%m-%d')
switcher ={
        0 : "Lunes",
        1 : "Martes",
        2 :"Miercoles",
        3 :"Jueves",
        4 :"Viernes",
        5 :"Sabado",
        6 :"Domingo"
    }
df['Dia_semana_inicio'] = list(map(lambda day_week : switcher.get(day_week.weekday(), "otro"),df.FechaInicio))
df['Dia_semana_hecho']  = list(map(lambda day_week : switcher.get(day_week.weekday(), "otro"),df.FechaHecho))


## Stopwords

In [67]:
df['Delito'] = df['Delito'].str.replace(r'\(|\)', '', regex=True)

In [68]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
# Función para eliminar stop words
def remove_stopwords(text):
    words = text.split()
    clean_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(clean_words)

In [70]:
# Aplicar la función a la columna 'texto'
df['Delito'] = df['Delito'].apply(remove_stopwords)

In [71]:
df.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,dia_inicio,dia_hecho,horas_hecho,minutos_hecho,segundos_hecho,horas_inicio,minutos_inicio,segundos_inicio,Dia_semana_inicio,Dia_semana_hecho
0,8324429,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62,FISICA,OFENDIDO,...,4,29,12,0,0,12,19,0,Viernes,Miercoles
1,8324430,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,...,4,15,15,0,0,12,20,0,Viernes,Sabado
2,8324431,2019,Enero,2019-01-04,ROBO TRANSEUNTE SALIENDO BANCO VIOLENCIA,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,...,4,22,15,30,0,12,23,0,Viernes,Sabado
3,8324435,2019,Enero,2019-01-04,ROBO VEHICULO SERVICIO PARTICULAR VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,...,4,4,6,0,0,12,27,0,Viernes,Viernes
4,8324438,2019,Enero,2019-01-04,ROBO MOTOCICLETA VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,38,FISICA,VICTIMA,...,4,3,20,0,0,12,35,0,Viernes,Jueves


## Particion por años

In [72]:
#Dividir el dataframe en 2 : (2019-2022) y (2023)
delitos = df[df["Año_inicio"]!=2023]
delitos_2023 = df[df["Año_inicio"]==2023]

In [73]:
delitos_2023.drop(['Categoria'], axis=1)

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,...,dia_inicio,dia_hecho,horas_hecho,minutos_hecho,segundos_hecho,horas_inicio,minutos_inicio,segundos_inicio,Dia_semana_inicio,Dia_semana_hecho
0,9361878,2023,Enero,2023-01-01,LESIONES CULPOSAS TRANSITO VEHICULAR COLISION,Masculino,38,MORAL,OFENDIDO,FUERO COMUN,...,1,31,22,8,0,0,9,0,Domingo,Sabado
1,9361879,2023,Enero,2023-01-01,ROBO TRANSEUNTE VIA PUBLICA VIOLENCIA,Masculino,50,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,1,31,23,50,0,0,41,0,Domingo,Sabado
2,9361880,2023,Enero,2023-01-01,ROBO VEHICULO SERVICIO PARTICULAR VIOLENCIA,Masculino,34,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,1,31,20,30,0,0,44,0,Domingo,Sabado
3,9361881,2023,Enero,2023-01-01,LESIONES CULPOSAS TRANSITO VEHICULAR COLISION,Masculino,38,FISICA,LESIONADO,FUERO COMUN,...,1,31,23,30,0,0,47,0,Domingo,Sabado
5,9361883,2023,Enero,2023-01-01,ROBO OBJETOS,Masculino,38,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,1,31,23,15,0,1,4,0,Domingo,Sabado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64339,9429532,2023,Marzo,2023-03-31,"ROBO S/V DENTRO NEGOCIOS, AUTOSERVICIOS, CONVE...",Femenino,69,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,31,31,7,30,0,23,48,0,Viernes,Viernes
64340,9429533,2023,Marzo,2023-03-31,VIOLENCIA FAMILIAR,Femenino,17,FISICA,VICTIMA,FUERO COMUN,...,31,30,22,0,0,23,49,0,Viernes,Jueves
64341,9429534,2023,Marzo,2023-03-31,ROBO OBJETOS INTERIOR VEHICULO,Femenino,43,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,31,31,7,0,0,23,51,0,Viernes,Viernes
64342,9429535,2023,Marzo,2023-03-31,ROBO OBJETOS,Femenino,39,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,31,31,10,1,0,23,54,0,Viernes,Viernes


# Modelo

## CountVectorizer

In [74]:
# Separar características de texto y numéricas
X_text = delitos['Delito']
X_2023 = delitos_2023['Delito']
y = delitos['Categoria']

# Crear matriz de frecuencia de términos
vectorizer = CountVectorizer(max_features=100)
X_text = vectorizer.fit_transform(X_text)
X_2023 = vectorizer.fit_transform(X_2023)

In [75]:
columns = vectorizer.get_feature_names()
X_text = pd.DataFrame(X_text.toarray(), columns=columns)


In [76]:
X = X_text

## Train, Test, validation script

In [77]:
# Dividir conjunto de datos en entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


## Model training

In [78]:
# Entrenar modelo de clasificación
clf = MultinomialNB()
clf.fit(X_train, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Model evaluation

In [79]:
# Evaluar modelo en conjunto de val
score = clf.score(X_val, y_val)
print('Accuracy:', score)


Accuracy: 0.9033420397525075


## Model labeling

Juntarle al datafrane de 2023 la prediccion y_pred 

In [80]:
y_pred = clf.predict(X_2023)

In [81]:
# Agregarle al 2023 la categoria que arrojó el modelo 
delitos_2023['Categoria'] = y_pred 

In [82]:
# Unir datos  del (2019-2022) con los datos del 2023 
df =pd.concat([delitos, delitos_2023])
df

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,dia_inicio,dia_hecho,horas_hecho,minutos_hecho,segundos_hecho,horas_inicio,minutos_inicio,segundos_inicio,Dia_semana_inicio,Dia_semana_hecho
0,8324429,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62,FISICA,OFENDIDO,...,4,29,12,0,0,12,19,0,Viernes,Miercoles
1,8324430,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,...,4,15,15,0,0,12,20,0,Viernes,Sabado
2,8324431,2019,Enero,2019-01-04,ROBO TRANSEUNTE SALIENDO BANCO VIOLENCIA,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,...,4,22,15,30,0,12,23,0,Viernes,Sabado
3,8324435,2019,Enero,2019-01-04,ROBO VEHICULO SERVICIO PARTICULAR VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,...,4,4,6,0,0,12,27,0,Viernes,Viernes
4,8324438,2019,Enero,2019-01-04,ROBO MOTOCICLETA VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,38,FISICA,VICTIMA,...,4,3,20,0,0,12,35,0,Viernes,Jueves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64339,9429532,2023,Marzo,2023-03-31,"ROBO S/V DENTRO NEGOCIOS, AUTOSERVICIOS, CONVE...",DELITO DE BAJO IMPACTO,Femenino,69,FISICA,VICTIMA Y DENUNCIANTE,...,31,31,7,30,0,23,48,0,Viernes,Viernes
64340,9429533,2023,Marzo,2023-03-31,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,17,FISICA,VICTIMA,...,31,30,22,0,0,23,49,0,Viernes,Jueves
64341,9429534,2023,Marzo,2023-03-31,ROBO OBJETOS INTERIOR VEHICULO,SECUESTRO,Femenino,43,FISICA,VICTIMA Y DENUNCIANTE,...,31,31,7,0,0,23,51,0,Viernes,Viernes
64342,9429535,2023,Marzo,2023-03-31,ROBO OBJETOS,HECHO NO DELICTIVO,Femenino,39,FISICA,VICTIMA Y DENUNCIANTE,...,31,31,10,1,0,23,54,0,Viernes,Viernes


In [83]:
# Se pasó la columna de categoría al final del dataframe 
column2 = df.pop('Categoria')
df['Categoria'] = column2

### Agregar columnas de retrato para posteriormente usarlas con Rekognition

In [84]:
#Simulamos retrato hablado generando un random entre 1 a 13 (#de retratos con los que contamos)
random_numbers = [random.randint(1, 13) for _ in range(len(df))]
# Agregar la columna de números aleatorios al DataFrame
df['retrato'] = random_numbers


# Código para detectar sospechosos con el modelo de rekognition

## Obtener las rutas para todas las imágenes del S3 

In [85]:
s3 = boto3.resource('s3')
bucket_name = "itam-analytics-alberto"
bucket_prefix_ret = "delitos/img/retratos"
bucket_prefix_sos = "delitos/img/sospechosos"

my_bucket = s3.Bucket(bucket_name)

ret = []
sos =[]

# Retratos 
for my_bucket_object in my_bucket.objects.filter(Prefix=bucket_prefix_ret):
    # Obtener los nombres de las rutas para todos los retratos  
    ret.append(my_bucket_object.key)


ret.pop(0) 

# Sospechosos
for my_bucket_object in my_bucket.objects.filter(Prefix=bucket_prefix_sos):
   # Obtener los nombres de las rutas para todos los sospechosos
     sos.append(my_bucket_object.key)


sos.pop(0)   


'delitos/img/sospechosos/'

In [86]:
rekognition = boto3.client('rekognition',
                           aws_access_key_id='SECRET', 
                           aws_secret_access_key='SECRET', region_name='us-east-1')

In [87]:
# Crear un diccionario en el que la llave sean todos los retratos posibles 
all_retratos = df.retrato.unique()
ret_sos_relation = {key: None for key in all_retratos}
print(ret_sos_relation)

{11: None, 9: None, 6: None, 8: None, 5: None, 7: None, 2: None, 13: None, 4: None, 1: None, 10: None, 12: None, 3: None}


In [88]:
for i in range(0,len(ret),1):
    best_similarity= 0
    for j in range(0,len(sos),1):
        # Carga las imágenes a comparar
        response = rekognition.compare_faces(
            SourceImage={
                 'S3Object': {
                     'Bucket': 'itam-analytics-alberto',
                     'Name':   ret[i]
                             }},
            TargetImage={
                'S3Object': {
                     'Bucket': 'itam-analytics-alberto',
                     'Name':   sos[j]
                            }},
            SimilarityThreshold=0  # Puedes ajustar este valor según tu caso
        )

        # Obtiene la puntuación de similitud
        similarity = response['FaceMatches'][0]['Similarity']
        if similarity >= best_similarity:
            best_similarity = similarity
            # LLenar el diccionario de retratos con los sospechosos asignados y el nivel de similitud 
            key = int(re.findall("\d+", ret[i])[0]) 
            bestImgSimil = int(re.findall("\d+", sos[j])[0]) 
            ret_sos_relation[key] = [ bestImgSimil,similarity]
            

In [89]:
# Ver el diccionario lleno con retratos, sospechosos y porcentaje de similitud 
print(ret_sos_relation)

{11: [11, 99.96415710449219], 9: [9, 2.584242820739746], 6: [10, 3.0010321140289307], 8: [8, 1.1975784301757812], 5: [5, 14.961284637451172], 7: [7, 15.112532615661621], 2: [2, 70.27783203125], 13: [13, 94.7482681274414], 4: [4, 79.49003601074219], 1: [1, 85.18539428710938], 10: [10, 6.86883020401001], 12: [12, 99.66559600830078], 3: [7, 2.0562756061553955]}


In [90]:
# Crear las columnas de sospechosos y porcentaje de similitud 
df['sospechoso'] = np.nan
df['porcentaje_similitud'] = np.nan

In [91]:
df['sospechoso'] =df['retrato'].apply(lambda x: ret_sos_relation[x][0])
df['porcentaje_similitud'] = df['retrato'].apply(lambda x: ret_sos_relation[x][1])


# Send clean data to S3 

In [92]:
cols = ['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica', 'competencia',
       'Año_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho', 'HoraInicio',
       'alcaldia_hechos', 'colonia_datos', 'fgj_colonia_registro', 'latitud',
       'longitud', 'dia_inicio', 'dia_hecho', 'horas_hecho', 'minutos_hecho',
       'segundos_hecho', 'horas_inicio', 'minutos_inicio', 'segundos_inicio',
       'Dia_semana_inicio', 'Dia_semana_hecho', 'Categoria', 'retrato',
       'sospechoso', 'porcentaje_similitud']
df = df[cols]

In [93]:
s3 = boto3.resource('s3')
bucket_name = "itam-analytics-alberto"
my_bucket = s3.Bucket(bucket_name)

df.to_csv('s3://itam-analytics-alberto/delitos/cleanData/delitos.csv', index=False)

In [94]:
df.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,...,segundos_hecho,horas_inicio,minutos_inicio,segundos_inicio,Dia_semana_inicio,Dia_semana_hecho,Categoria,retrato,sospechoso,porcentaje_similitud
0,8324429,2019,Enero,2019-01-04,FRAUDE,Masculino,62,FISICA,OFENDIDO,FUERO COMUN,...,0,12,19,0,Viernes,Miercoles,DELITO DE BAJO IMPACTO,11,11,99.964157
1,8324430,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",Femenino,38,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,0,12,20,0,Viernes,Sabado,DELITO DE BAJO IMPACTO,9,9,2.584243
2,8324431,2019,Enero,2019-01-04,ROBO TRANSEUNTE SALIENDO BANCO VIOLENCIA,Masculino,42,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,0,12,23,0,Viernes,Sabado,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,6,10,3.001032
3,8324435,2019,Enero,2019-01-04,ROBO VEHICULO SERVICIO PARTICULAR VIOLENCIA,Masculino,35,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,...,0,12,27,0,Viernes,Viernes,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,8,8,1.197578
4,8324438,2019,Enero,2019-01-04,ROBO MOTOCICLETA VIOLENCIA,Masculino,38,FISICA,VICTIMA,FUERO COMUN,...,0,12,35,0,Viernes,Jueves,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,6,10,3.001032
